In [25]:
import requests
import bs4

res = requests.get("https://tfl.gov.uk/tube-dlr-overground/status/")
soup = bs4.BeautifulSoup(res.text, "lxml")

In [26]:
district_info = soup.select(".rainbow-list-item.district")

In [4]:
for item in district_info:
    print(item.text)




District



Part suspended 


 



District Line: No service between Ealing Common to Ealing Broadway while the police carry out an investigation. GOOD SERVICE on the rest of the line 




Replan your journey


Close status




In [5]:
for item in district_info:
    text = item.text.replace('\n',' ')
    print(text)

   District    Part suspended         District Line: No service between Ealing Common to Ealing Broadway while the police carry out an investigation. GOOD SERVICE on the rest of the line      Replan your journey   Close status  


In [27]:
status = []
for item in district_info:
    status.append(item.text.replace("\n", " "))
status = status[0].split('Replan your journey')[0]
status

'   District    Good service     '

In [16]:
if 'Severe delays' in status:
        status = 'Severe delays' + status.split("Severe delays")[-1]

In [17]:
status

'Severe delays Earls Court to Wimbledon due to an earlier signal failure at Wimbledon Park. GOOD SERVICE on the rest of the line. London Underground tickets will be accepted on South Western Railway and local buses.      '

In [6]:
stations = ["South Kensington","Earls Court","West Brompton","Fulham Broadway", "Parsons Green","Putney Bridge","East Putney","Southfields","Wimbledon Park","Wimbledon"]

In [20]:
if any(station in status for station in stations):
    print(True)

True


In [1]:
status1 = "District    Part suspended         District Line: No service between Ealing Common to Ealing Broadway while the police carry out an investigation. GOOD SERVICE on the rest of the line      Replan your journey   Close status  "

In [2]:
status2 = "District    Severe delays         District Line: Severe delays Earls Court to Wimbledon due to an earlier signal failure at Wimbledon Park. GOOD SERVICE on the rest of the line. London Underground tickets will be accepted on South Western Railway and local buses."

In [3]:
status3 = '   District    Good service     '

In [ ]:
status4 = 

In [30]:
"Severe delays" in status2

True

In [4]:
def update_status(status, line, delay):
    if any(station in status for station in stations):
            status = status.split(" GOOD SERVICE")[0]
            status = f"{line.capitalize()} line has {delay.lower()}" + status.split(f"{delay}")[-1]
    elif "GOOD SERVICE" in status:
            status = f"{delay} on parts of {line} line. Your route has good service."
    else:
            status = f"{delay} on parts of {line} line. Your route may be affected."
    return status
    

def parse_status(status, line):
    if "Good service" in status:
        parsed_status =  f"Good service on whole {line} line."
    elif "Severe delays" in status:
        delay = "Severe delays"
        parsed_status = update_status(status, line, delay)
    elif "Minor delays" in status:
        delay = "Minor delays"
        parsed_status = update_status(status, line, delay)
    elif "Part suspended" in status:
        delay = "No service"
        parsed_status = update_status(status, line, delay)

    return parsed_status

In [7]:
parse_status(status2,"district")

'District line has severe delays Earls Court to Wimbledon due to an earlier signal failure at Wimbledon Park.'

In [8]:
parse_status(status1,"district")

'No service on parts of district line. Your route has good service.'

In [9]:
parse_status(status3,"district")

'Good service on whole district line.'

In [44]:
import requests
import bs4

res = requests.get("https://tfl.gov.uk/tube-dlr-overground/status/")
soup = bs4.BeautifulSoup(res.text, "lxml")
victoria_info = soup.select(".rainbow-list-item.victoria")
status = []
for item in victoria_info:
    status.append(item.text.replace("\n", " "))

In [45]:
status_new = status[0].split("Replan your journey")[0]
status_new

'   Victoria    Part suspended  Severe delays    \xa0    Victoria Line: No service between Warren Street and Victoria while we fix a track fault at Oxford Circus. SEVERE DELAYS on the rest of the line. London Underground tickets are being accepted on London Buses, London Overground, South Eastern Trains, Great Northern, Greater Anglia and South Western Railway. Customers are advised to avoid the Victoria line due to emergency engineering work at Oxford Circus .     '

In [46]:
import re

def update_status_new(status, line, stations, delay):
    if any(station in status for station in stations):
        status = status.split(" GOOD SERVICE")[0]
        status = (
            f"{line.capitalize()} line has {delay.lower()}"
            + status.split(f"{delay}")[-1]
        )
    elif "GOOD SERVICE" in status:
        status = f"{delay} on parts of {line} line. Your route has good service."
    elif delay == "Line suspended":
        status = f"{line.capitalize()} line suspended. Find an alternative route."
    else:
        status = f"{delay} on parts of {line} line. Your route may be affected."
    return status

def update_status_suspended(status, line, stations, delay):
    if any(station in status for station in stations):
        status = re.split(" (MINOR|SEVERE) DELAYS", status)[0]
        status = status.split(f"{line.capitalize()} Line:")[1]
        status = (
            f"{line.capitalize()} line is part suspended. "
            + status.split(f"{delay}")[-1].strip() + f" {delay} on rest of line."
        )
    else:
        status = f"{line.capitalize()} line is part suspended. {delay} on rest of line. Your route may be affected."
    return status
    
    
def parse_status_new(status, line, stations):
    if "Part suspended" in status:
        if "GOOD SERVICE" in status:
            delay = "Good service"
            parsed_status = update_status_new(status, line, stations, delay)
        elif "MINOR DELAYS" in status:
            delay = "Minor delays"
            parsed_status = update_status_suspended(status, line, stations, delay)
        elif "SEVERE DELAYS" in status:
            delay = "Severe delays"
            parsed_status = update_status_suspended(status, line, stations, delay)
    elif "Severe delays" in status:
        delay = "Severe delays"
        parsed_status = update_status_new(status, line, stations, delay)
    elif "Minor delays" in status:
        delay = "Minor delays"
        parsed_status = update_status_new(status, line, stations, delay)
    elif "Good service" in status:
        parsed_status = f"Good service on whole {line} line."
    else:
        delay = "Line suspended"
        parsed_status = update_status_new(status, line, stations, delay)

    return parsed_status

In [47]:
print(parse_status_new(status_new, 'victoria',["Victoria", "Pimlico", "Vauxhall", "Stockwell"]))

Victoria line is part suspended. No service between Warren Street and Victoria while we fix a track fault at Oxford Circus. Severe delays on rest of line.


In [42]:
metro_info = soup.select(".rainbow-list-item.metropolitan")
status = []
for item in metro_info:
    status.append(item.text.replace("\n", " "))
status_new = status[0].split("Replan your journey")[0]
status_new

'   Metropolitan    Part suspended  Severe delays    \xa0    Metropolitan Line: No service between Wembley Park and Aldgate while we fix a signal failure at Baker Street. SEVERE DELAYS on the rest of the line. London Underground tickets are being accepted on London Buses.      '

In [43]:
print(parse_status_new(status_new, 'metropolitan',["Victoria", "Pimlico", "Vauxhall", "Stockwell"]))

Metropolitan line is part suspended. Severe delays on rest of line. Your route may be affected.
